<a href="https://colab.research.google.com/github/ua-datalab/Geospatial_Workshops/blob/main/notebooks/detecto_geospatial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning with Detecto

This notebook uses [detecto](https://detecto.readthedocs.io/en/latest/), a lightweight Python library that will allow us to create a convolutional neural network object detection model in very few lines of code. It downloads a pre-trained Faster R-CNN ResNet50 FPN from Pytorch as the starting point. However, the pre-trained model is not specifically trained to identify objects from aerial imagery. We have to train it to do so. In this notebook, we will use an existing labeled training dataset to fine-tune the model to identify 10 different objects from aerial imagery.

In [ ]:
# install the required libraries to the virtual machine (! indicates install to underlying computer)
!pip install detecto --quiet
!pip install pascal-voc-writer --quiet

### Run the pre-trained model on some test imagery

In general, the pre-trained model should NOT be able to identify objects very well from an aerial view. It has not yet been trained to do so.

In [ ]:
##Get a test image and run the pre-trained model on it. It should NOT produce good results.

#Import necessary modules from Detecto python library
from detecto.core import Model #bring in the Faster R-CNN ResNet50 FPM model
from detecto.utils import read_image
from detecto.visualize import show_labeled_image

!wget https://raw.githubusercontent.com/ua-datalab/Geospatial_Workshops/main/data/vehicle_test2.png
image = read_image('vehicle_test2.png')

model = Model() #assign the pre-trained model to a variable
labels, boxes, scores = model.predict(image)
show_labeled_image(image, boxes, labels)

In [ ]:
#Get a test image and run the pre-trained model on it. It should NOT produce good results.

!wget https://raw.githubusercontent.com/ua-datalab/Geospatial_Workshops/main/data/baseball_field_test.png

image = read_image('baseball_field_test.png')
labels, boxes, scores = model.predict(image)
show_labeled_image(image, boxes, labels)

### Get Training Dataset
The [NWPU VHR-10 dataset](https://github.com/chaozhong2010/VHR-10_dataset_coco) is a very high resolution (VHR) aerial imagery dataset that consists of 800 total images. The dataset has ten classes of labeled objects: 1. airplane(757), 2. ship(302), 3. storage tank(655), 4. baseball diamond(390), 5. tennis court(524), 6. basketball court(159), 7. ground track field(163), 8. harbor(224), 9. bridges(124), and 10. vehicle(477).

In [ ]:
#Download the labeled training data set (NWPU_VHR-10.tar) from github. It will be located in the `/content` directory in Colab
!wget https://github.com/ua-datalab/Geospatial_Workshops/raw/main/data/NWPU_VHR-10.tar

### The next line of code will decompress the tar file and give you access to the imagery and labels.

Within the root directory (NWPU_VHR-10_dataset), the subdirectory *positive_image_set* has 650 jpg images with each image containing at least one target object. The subdirectory *ground_truth* contains a a text file corresponding to each image. Each line of those text files defines a ground truth bounding box in the following format:
(x1,y1),(x2,y2),a

where (x1,y1) denotes the top-left coordinate of the bounding box, (x2,y2) denotes the right-bottom coordinate of the bounding box, and a is the object class (1-airplane, 2-ship, 3-storage tank, 4-baseball diamond, 5-tennis court, 6-basketball court, 7-ground track field, 8-harbor, 9-bridge, 10-vehicle).

In [ ]:
#Decompress the labeled training data tar file
!tar -xvf NWPU_VHR-10.tar

### Convert labeled dataset to Pascal VOC Format

**detecto** requires that the dataset is labeled in the [Pascal VOC format](http://host.robots.ox.ac.uk/pascal/VOC/). In this dataset, the labels for the images are stored in plain text files, so we will first convert the data to the required format.


In [ ]:
#Import the required tools

import os # used for looking through directories
import re # search in strings
from pascal_voc_writer import Writer # create the .xml files with the VOC format
from PIL import Image # used to get height, width of images

In [ ]:
# get a list of the text file names
base = './NWPU_VHR-10_dataset'
txt_files = os.listdir(os.path.join(base, 'ground_truth'))

In [ ]:
#Create a dictionary that maps the class number to the class name.

classes = {
    1: 'airplane',
    2: 'ship',
    3: 'storage tank',
    4: 'baseball diamond',
    5: 'tennis court',
    6: 'basketball court',
    7: 'ground track field',
    8: 'harbor',
    9: 'bridge',
    10: 'vehicle'
}

print(type(classes))

In [ ]:
# loop through each ground truth text file and convert to Pascal VOC format.
# This will create a new xml file for each image and will save it in the directory 'positive_image_set'
for txt in txt_files:
    # open the text file for reading
    with open(os.path.join(base, 'ground_truth', txt), 'r') as reader:

        # locate and open corresponding image
        img_id, _ = txt.split('.')
        img_path = os.path.join(base, 'positive_image_set', f'{img_id}.jpg')
        img = Image.open(img_path)

        # create a writer for the xml file
        xml_path = os.path.join(base, 'positive_image_set', f'{img_id}.xml')
        writer = Writer(img_path, img.width, img.height)

        # convert the text file to an xml file and save
        lines = reader.readlines()
        for line in lines:
            numbers = re.findall(r'\d+', line)
            if not numbers: # skip empty lines
                continue
            xmin, ymin, xmax, ymax, class_id = list(map(int, numbers))
            writer.addObject(classes[class_id], xmin, ymin, xmax, ymax)
        writer.save(xml_path)

### Load training images into Datasets
Next, we load the 650 images into 'datasets'


In [ ]:
# Import necessary module
from detecto.core import Dataset

# create the datasets
dataset = Dataset(os.path.join(base, 'positive_image_set/'))

### Create a random split of training and validation data from the dataset

In [ ]:
#import necessary modules from pytorch
import torch
from torch.utils.data import random_split

# this sets the seed of the random generator to ensure reproducibility
random_generator = torch.Generator().manual_seed(42)

# Split the dataset into 80% training and 20% validation
train_dataset, val_dataset = random_split(dataset, [0.8, 0.2])

### Display the training images and labels
**detecto** provides a visualization function that will display the image along with bounding boxes and labels for the objects it contains.

In [ ]:
from detecto.visualize import show_labeled_image

# show some of the training data. Change the [number] to other numbers to see other images.
image, targets = train_dataset[509]
show_labeled_image(image, targets['boxes'], targets['labels'])

In [ ]:
# we can also visualize the validation dataset
image, targets = val_dataset[1]
show_labeled_image(image, targets['boxes'], targets['labels'])

## Train the Model
With the data loaded, we can now train the model. Here, we use a DataLoader to wrap the Dataset object. This allows us to specify the batch size used during each training step, one of a few hyperparameters we can play with during fine tuning.

Note that we are required to tell **detecto** which labels we are interested in learning to identify.

In [ ]:
#import necessary modules
from detecto.core import DataLoader, Model

# wrap the training set in a DataLoader
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# specify all unique labels you're trying to predict
labels = [
    'airplane',
    'ship',
    'storage tank',
    'baseball diamond',
    'tennis court',
    'basketball court',
    'ground track field',
    'harbor',
    'bridge',
    'vehicle'
]

In [ ]:
##TRAINING RUN!!
# initialize a model with the target labels and fit the model
# This step will take some time and is accelerated with GPU
model = Model(labels)
losses = model.fit(train_loader,
                   val_dataset,
                   epochs=8,
                   lr_step_size=5,
                   learning_rate=0.001,
                   verbose=True)

Here we perform a sanity check to ensure that the training process is working correctly by checking that the loss is decreasing.

In [ ]:
from matplotlib import pyplot as plt

plt.plot(losses)
plt.show()

## Calculate Intersection over Union (IOU) to assess 'accuracy' of object detection.


In [ ]:
#The following code block will calculate IOU for each validation/prediction pair.
#A thresold of 0.5 has been set where an IOU value above 0.5 is considered a good prediction
!pip install torchvision
from torchvision.ops import box_iou
from detecto.core import Dataset
from detecto.utils import normalize_transform
from detecto.visualize import show_labeled_image
from tqdm import tqdm

def calculate_accuracy(model, val_dataset, iou_threshold=0.5):
  """
  Calculates the accuracy of the model on the validation dataset using torchvision.ops.box_iou.

  Args:
      model: The trained detecto model.
      val_dataset: The validation dataset.
      iou_threshold: The IoU threshold for considering a prediction correct.

  Returns:
      The accuracy of the model.
  """
  correct_predictions = 0
  total_predictions = 0

  for image, targets in tqdm(val_dataset):
      # Convert target boxes to a PyTorch tensor
      target_boxes = targets['boxes']

      labels, boxes, scores = model.predict(image)

      # Calculate IoU for all pairs of boxes
      iou_matrix = box_iou(target_boxes, boxes)

      for i in range(len(target_boxes)):
          gt_label = targets['labels'][i]

          # Get the IoU for the current ground truth box with all predicted boxes
          iou_row = iou_matrix[i]

          # Find the best matching predicted box
          best_iou, best_idx = torch.max(iou_row, dim=0)

          if best_iou >= iou_threshold:
              pred_label = labels[best_idx]
              if pred_label == gt_label:
                  correct_predictions += 1

          total_predictions += 1

  return correct_predictions / total_predictions if total_predictions > 0 else 0

# Example usage (assuming you have model and val_dataset defined)
accuracy = calculate_accuracy(model, val_dataset)
print("Accuracy:", accuracy)

## Save model to Colab disk

In [ ]:
#save model Colab disk
model.save('detecto_vhr_finetune.pth')

## Skip the Training, and use an existing fine-tuned model

If you have computation limitation and want to skip the model training, we can use a pre-trained model that has already been trained on the NWPU-VHR dataset. Download the model from huggingface.

In [ ]:
#Download the fine-tuned model from Hugging Face
!wget https://huggingface.co/jgillan/detecto_vhr_finetune/resolve/main/detecto_vhr_finetune.pth


from detecto.core import Model

#Load fine-tuned model into our session
model = Model.load('detecto_vhr_finetune.pth', labels)

## Visually Look at Validation Results

In [ ]:
images = [] #create an empty list
for i in range(8,12): #  create a for loop that will grab 4 images from the validation dataset
    image, _ = val_dataset[i]
    images.append(image)

top_predictions= model.predict_top(images) # Predict objects on the 4 validation images

In [ ]:
#Display 4 validation images and show the predicted bounding box and label.

from detecto.visualize import plot_prediction_grid

plot_prediction_grid(model, images, dim=(2, 2), figsize=(8, 8))

## Test the fine-tuned model on independent images

In [ ]:
#import necessary modules
import urllib.request
from PIL import Image
import IPython.display as display

# Replace the URL with the actual link to your .png file
url = 'https://raw.githubusercontent.com/ua-datalab/Geospatial_Workshops/main/data/airplane_test.png'
urllib.request.urlretrieve(url, 'airplane_test.png')

# Load and display the image
image = Image.open('airplane_test.png')
display.display(image)


In [ ]:
## Use the fine-tuned model on the independent image
image = read_image('airplane_test.png')
labels, boxes, scores = model.predict(image)
show_labeled_image(image, boxes, labels)

In [ ]:
## Use the fine-tuned model on the independent image

!wget https://raw.githubusercontent.com/ua-datalab/Geospatial_Workshops/main/data/vehicle_test2.png

image = read_image('vehicle_test2.png')
labels, boxes, scores = model.predict(image)
show_labeled_image(image, boxes, labels)

In [ ]:
## Use the fine-tuned model on the independent image

!wget https://raw.githubusercontent.com/ua-datalab/Geospatial_Workshops/main/data/baseball_field_test.png

image = read_image('baseball_field_test.png')
labels, boxes, scores = model.predict(image)
show_labeled_image(image, boxes, labels)